## HW: Decision Tree with ID3 Algorithm

### Introduction

In this homework, you will implement a simple ``ID3 Decision Tree`` method that we introduce in the lecture.

Given a dataset, where each row is a mushroom, and each column contains ``features`` (or called ``attributes``) of the mushroom. **The goal is to predict whether the mushroom is poisonous or not (i.e., poisonous or edible) based on features.** Note that the first column is the class of ``poisonous`` or ``edible``.

The mushroom dataset is taken from UCI data repository. The value of each feature/attribute column (from the second column) is categorical or ordinal. Each feature/attribute value is an abbreviation. You can refer to the detailed description for each feature/attribute value abbreviation from [UCI Mushroom Data Set](https://archive.ics.uci.edu/ml/datasets/mushroom). Below we copy and paste the official description.

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like "leaflets three, let it be" for Poisonous Oak and Ivy.


Feature/Attribute Information:

1. cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
4. bruises?: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d


In [1]:
import pandas as pd
df_shroom = pd.read_csv('D:\程設\mushrooms.csv')
df_shroom

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,poisonous,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,edible,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,edible,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,poisonous,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,edible,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,edible,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,edible,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,edible,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,poisonous,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


### Entropy

The decision tree algorithm works by looking at all the features, and picking the 'best' one to split the data on, then running recursively on the split data.

The way we pick the 'best' feature to split on is by picking the feature that decreases 'entropy'.

What's entropy? Recall that we're trying to classify mushrooms as poisonous or not. Entropy is a value that will be low if a group of mushrooms mostly has the same class (all poisonous or all edible) and high if a group of mushrooms varies in their classes (half poisonous and half edible). So every split of the data that minimizes entropy is a split that does a good job classifying.

In [3]:
def entropy(probs):
    ### take a list of probabilities and calculate their entropy values
    
    # your code here
    import math 
    list_entropy= sum( [-prob*math.log(prob, 2) for prob in probs] )
    
    return list_entropy
    

def entropy_of_list(a_list):
    ### take a list of items with discrete values (e.g., poisonous, edible)
    ### and return the list of entropy values for those items
    
    # your code here
    from collections import Counter     
    cnt = Counter(x for x in a_list)      
    num_instances = len(a_list)*1.0     
    probs = [x / num_instances for x in cnt.values()]
    
    return entropy(probs)
    
# the initial entropy of the poisonous/edible column for all instances in the dataset
total_entropy = entropy_of_list(df_shroom['class'])
print(total_entropy)

0.9990678968724604


In order to decide which feature to split on, we want to quantify how each feature decreases the entropy. 

We do this in a fairly intuitive way: we split our dataset by the possible values of an feature, then do a weighted sum of the entropies for each of these split datasets, weighted by how big that sub-dataset is. 

We'll create a function that quantifies the decrease in entropy, or conversely, the ``information gain``.

In [12]:
def information_gain(df, split_feature_name, target_feature_name):
    
    ### take a dataFrame of features, and quantify the entropy of a target feature
    ### after performing a split along the values of another feature
    ### and calculate the corresponding information gain

    # your code here
    df_split = df.groupby(split_feature_name) 
    for name,group in df_split:         
        print(name)         
        print(group)
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({target_feature_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_feature_name] 
    df_agg_ent.columns = ['Entropy', 'PropObservations'] 
    new_entropy = sum(df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_feature_name]) 
    info_gain= old_entropy-new_entropy
    return info_gain

print('\nExample: information gain for the best feature is ' + str(information_gain(df_shroom, 'odor', 'class')))

a
       class cap-shape cap-surface cap-color bruises odor gill-attachment  \
1     edible         x           s         y       t    a               f   
5     edible         x           y         y       t    a               f   
6     edible         b           s         w       t    a               f   
9     edible         b           s         y       t    a               f   
11    edible         x           y         y       t    a               f   
...      ...       ...         ...       ...     ...  ...             ...   
2018  edible         f           f         w       t    a               f   
2045  edible         x           y         y       t    a               f   
2051  edible         x           y         w       t    a               f   
2059  edible         x           s         w       t    a               f   
2092  edible         b           s         w       t    a               f   

     gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
1

### ID3 Decision Tree Algorithm

Now you will write the decision tree algorithm itself, called ``ID3``.

HINT: you can utilize ``nested dictionary`` to implement the tree.

In [17]:
def id3(df, target_feature_name, feature_names, default_class = None):
    
    ## counting for the target feature
    from collections import Counter
    cnt = Counter(x for x in df[target_feature_name])
    
    ## First check: do instances in this split of the dataset belong to the same class?
    # (i.e., all mushrooms in this set are poisonous)
    # if yes, return that homogenous label (e.g., 'poisonous')
    if len(cnt) == 1:
        return list(cnt.keys())[0]
    
    ## Second check: is this split of the dataset empty?
    # if yes, return a default value
    elif df.empty or (not feature_names):
        return default_class 
    
    ## Otherwise: this dataset is ready to be split!
    else:
        ### step 1: get the default value for next recursive call of this function
        index_of_max = list(cnt.values()).index(max(cnt.values())) 
        default_class = list(cnt.keys())[index_of_max] # most common value of target feature in dataset
        
        ### step 2: choose the best feature to split on
        
        # your code here 
        best_attr = feature_names[index_of_max]
        
        ### step 3: create an empty tree, to be populated in a moment
        
        # your code here
        tree = {best_attr:{}}
        
        
        ### Step 4: split dataset
        # on each split, recursively call this "id3" function
        # populate the empty tree with subtrees, which
        # are the result of the recursive call
        
        # your code here
        remaining_feature_names = [i for i in feature_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr): 
            subtree = id3(data_subset,
                          target_feature_name,                         
                          remaining_feature_names,                         
                          default_class)
            tree[best_attr][attr_val] = subtree
        
        return tree

In [18]:
# get feature names (all but 'class' column)
feature_names = list(df_shroom.columns)
feature_names.remove('class')

In [19]:
# visualize the constructed decision tree using pprint package
from pprint import pprint
# use all instances (df_shroom) to build the decision (suppose all instances are for training)
tree = id3(df_shroom, 'class', feature_names)
pprint(tree)

{'cap-surface': {'f': {'cap-shape': {'b': {'bruises': {'f': {'odor': {'n': {'gill-attachment': {'f': {'gill-spacing': {'c': 'poisonous',
                                                                                                                       'w': 'edible'}}}}}}}},
                                     'f': {'cap-color': {'c': 'edible',
                                                         'e': 'edible',
                                                         'g': {'bruises': {'f': {'gill-attachment': {'f': {'gill-spacing': {'c': {'gill-size': {'b': 'poisonous',
                                                                                                                                                'n': 'edible'}},
                                                                                                                            'w': 'edible'}}}},
                                                                           't': 'edible'}},
                   

### Classification Accuracy: Entire Datast as Training Data and Testing Data

Let's make sure the resulting tree accurately predicts the class, based on the features.

Below is a ``classify`` algorithm that takes an instance and classifies it based on the decision tree.

In [20]:
def classify(instance, tree, default = None):
    feature = list(tree.keys())[0]
    if instance[feature] in list(tree[feature].keys()):
        result = tree[feature][instance[feature]]
        if isinstance(result, dict): # this is a tree, delve deeper
            return classify(instance, result)
        else:
            return result # this is a label
    else:
        return default

In [21]:
df_shroom['predicted'] = df_shroom.apply(classify, axis=1, args=(tree,'poisonous')) 
    # classify func allows for a default arg: when tree does not have a prediction result for a particular
    # combitation of feature-values, we can use 'poisonous' as the default prediction (better safe than sorry!)

print('Accuracy on all instances is ' + str(sum(df_shroom['class'] == df_shroom['predicted']) / (1.0*len(df_shroom.index))))

df_shroom[['class', 'predicted']]

Accuracy on all instances is 1.0


,class,predicted
0,poisonous,poisonous
1,edible,edible
2,edible,edible
3,poisonous,poisonous
4,edible,edible
...,...,...
8119,edible,edible
8120,edible,edible
8121,edible,edible
8122,poisonous,poisonous


### Classification Accuracy: Split the Dataset into Training and Testing Sets

A better evaluation the classification algorithm is to train it on a subset of the data (i.e., training data), then test it on a different subset (i.e., test data), where two subsets are disjoint.

In [22]:
training_data = df_shroom.iloc[1:-3000].copy() # all but last 3 thousand instances
test_data  = df_shroom.iloc[-3000:].copy() # just the last 3 thousand
train_tree = id3(training_data, 'class', feature_names)

test_data['predicted2'] = test_data.apply(                               # <---- test_data source
                                          classify, 
                                          axis=1, 
                                          args=(train_tree,'poisonous')) # <---- train_data tree

print('Accuracy on the test data is ' + str(sum(test_data['class']==test_data['predicted2'] ) / (1.0*len(test_data.index))))

Accuracy on the test data is 0.46166666666666667
